# Manditory Requirements

In [1]:
import pandas as pd 
import requests
from pymongo import MongoClient

def send_http_request(url: str, body=None, method="POST"):
    try:
        response = requests.request(method, url, json=body)
        # Check if the request was successful (status code 200)        
        if response.status_code == 200:
            content_type = response.headers.get('Content-Type')
            if content_type and 'application/json' in content_type:
                try:
                    return response.json()
                except requests.exceptions.JSONDecodeError as e1:
                    return response.text
            else:
                return response.text

        else:
            print(f"Request exited with status code {response.status_code}: {response.reason}")
    except requests.RequestException as e:
        return e
    return None

Constant mapping between exercise names and their root derivation trees:

In [2]:
name_to_model_ids = {
    'Courses': ['JDKw8yJZF5fiP3jv3', 'PSqwzYAfW9dFAa9im'],
    'ProductionLine_v2_v3': ['aTwuoJgesSd8hXXEP', 'bNCCf9FMRZoxqobfX'],
    'Train': ['QxGnrFQnXPGh2Lh8C'],
    'SocialNetwork': ['dkZH6HJNQNLLDX6Aj'],
    'TrashFOL': ['sDLK7uBCbgZon3znd'],
    'ClassroomFOL': ['YH3ANm7Y5Qe5dSYem'],
    'TrashRL': ['PQAJE67kz8w5NWJuM'],
    'ClassroomRL': ['zRAn69AocpkmxXZnW'],
    'Graphs': ['gAeD3MTGCCv8YNTaK'],
    'LTS': ['zoEADeCW2b2suJB2k'],
    'ProductionLine_v1': ['jyS8Bmceejj9pLbTW'],
    'CV': ['JC8Tij8o8GZb99gEJ'],
    'TrashLTL': ['9jPK8KBWzjFmBx4Hb']
}


# Setup Databases

## Populate Alloy4fun Database

In [ ]:
from populate_database import populate_database
populate_database()

## Setup HiGenA

In [ ]:
for (name,ids) in name_to_model_ids.items():
    send_http_request(url="http://localhost:8080/hint/higena-setup",  body=ids)

## Setup SpecAssistant

Optional SpecAssistant Database Drop

In [3]:
send_http_request(url="http://localhost:8080/hint/debug-drop-db", method="GET")

Request exited with status code 204: No Content


In [4]:
def send_specassistant_setup_from_name(name):
    if (name in name_to_model_ids):
        return send_http_request(url="http://localhost:8080/hint/specassistant-setup?prefix="+name, body=name_to_model_ids[name], method="GET")
    else:
        return "Unkown Exercise"

### Setup All

In [5]:
# WARNING: This takes some time and will run in the foreground, setting up every graph, one at a time
for name in name_to_model_ids.keys():
    print(send_specassistant_setup_from_name(name))

Setup completed for Courses with model_ids [JDKw8yJZF5fiP3jv3, PSqwzYAfW9dFAa9im]
Setup completed for ProductionLine_v2_v3 with model_ids [aTwuoJgesSd8hXXEP, bNCCf9FMRZoxqobfX]
Setup completed for Train with model_ids [QxGnrFQnXPGh2Lh8C]
Setup completed for SocialNetwork with model_ids [dkZH6HJNQNLLDX6Aj]
Setup completed for TrashFOL with model_ids [sDLK7uBCbgZon3znd]
Setup completed for ClassroomFOL with model_ids [YH3ANm7Y5Qe5dSYem]
Setup completed for TrashRL with model_ids [PQAJE67kz8w5NWJuM]
Setup completed for ClassroomRL with model_ids [zRAn69AocpkmxXZnW]
Setup completed for Graphs with model_ids [gAeD3MTGCCv8YNTaK]
Setup completed for LTS with model_ids [zoEADeCW2b2suJB2k]
Setup completed for ProductionLine_v1 with model_ids [jyS8Bmceejj9pLbTW]
Setup completed for CV with model_ids [JC8Tij8o8GZb99gEJ]
Setup completed for TrashLTL with model_ids [9jPK8KBWzjFmBx4Hb]


# Database Study

In [6]:
from a4f_mongo_pipelines import *

mongo_uri = "mongodb://localhost:27017/"
database_name = "meteor"

## Data Gathering

### Get GraphId Maps

In [7]:
client = MongoClient(mongo_uri)
db = client[database_name]

graph_collection = db["Graph"]

name_to_graph_ids = {} 

for doc in graph_collection.aggregate(get_graph_id_dict_pipeline()):
    name_to_graph_ids[doc["_id"]] = doc["graph_ids"]

client.close()

name_to_graph_ids

{'CV': [ObjectId('6676e4c1588c844b680d8711'),
  ObjectId('6676e4c1588c844b680d8712'),
  ObjectId('6676e4c1588c844b680d8713'),
  ObjectId('6676e4c1588c844b680d8714')],
 'TrashFOL': [ObjectId('6676e3dc588c844b680d50ba'),
  ObjectId('6676e3dc588c844b680d50bb'),
  ObjectId('6676e3dc588c844b680d50bc'),
  ObjectId('6676e3dc588c844b680d50bd'),
  ObjectId('6676e3dc588c844b680d50be'),
  ObjectId('6676e3dc588c844b680d50bf'),
  ObjectId('6676e3dc588c844b680d50c0'),
  ObjectId('6676e3dc588c844b680d50c1'),
  ObjectId('6676e3dc588c844b680d50c2')],
 'ProductionLine_v2_v3': [ObjectId('6676e2d6588c844b680ced51'),
  ObjectId('6676e2d6588c844b680ced52'),
  ObjectId('6676e2d6588c844b680ced53'),
  ObjectId('6676e2d6588c844b680ced54'),
  ObjectId('6676e2d6588c844b680ced55'),
  ObjectId('6676e2d6588c844b680ced56'),
  ObjectId('6676e2d6588c844b680ced57'),
  ObjectId('6676e2d6588c844b680ced58'),
  ObjectId('6676e2d6588c844b680ced59'),
  ObjectId('6676e2d6588c844b680ced5a')],
 'TrashRL': [ObjectId('6676e429588c

### Get Graph Stats Data Frames

In [8]:
client = MongoClient(mongo_uri)
db = client[database_name]

node_collection = db["Node"]

data = list(node_collection.aggregate(get_graph_node_statistics()))

graph_stats_df = pd.DataFrame(data)

client.close()

graph_stats_df

,name,valid_formulas,invalid_formulas,valid_submissions,invalid_submissions
0,CV-Inv1OK,7,42,77,118
1,CV-Inv2OK,16,63,52,98
2,CV-Inv3OK,6,109,26,172
3,CV-Inv4OK,1,197,1,314
4,ClassroomFOL-inv10OK,3,30,147,43
...,...,...,...,...,...
124,TrashRL-inv5Ok,10,75,155,193
125,TrashRL-inv6Ok,12,78,172,192
126,TrashRL-inv7Ok,25,108,135,268
127,TrashRL-inv8Ok,13,6,159,14


### Get Popular Node Data Frames

WARNING: Requires GraphId Maps

In [9]:
client = MongoClient(mongo_uri)
db = client[database_name]

node_collection = db["Node"]

name_to_pop_dfs = {}

for (name,graph_ids) in name_to_graph_ids.items():
    data = list(node_collection.aggregate(get_popular_nodes_pipeline(graph_ids)))[0:30] # Limits output to first 30 entries
    df_ = pd.DataFrame(data)
    name_to_pop_dfs[name] = df_

client.close()

name_to_pop_dfs

{'CV':     predicate                                            formula  frequency  \
 0   this/Inv1  ((User . (User <: visible)) in (User . (User <...         15   
 1   this/Inv4  (all ref0:(one User),ref1:(one Work)|((ref1 in...         13   
 2   this/Inv4  (all ref0:(one User),ref1:(one Work),ref2:(one...          8   
 3   this/Inv4  (((~ ((User <: visible) . (Work <: ids))) . ((...          7   
 4   this/Inv3  ((((User <: profile) . (Work <: ids)) . (~ ((U...          6   
 5   this/Inv2  (all ref0:(one User)|((Work . (ref0 <: (User <...          6   
 6   this/Inv3  (all ref0:(one Work),ref1:(one Work)|(((ref0 i...          5   
 7   this/Inv3  (all ref0:(one User),ref1:(one Work),ref2:(one...          5   
 8   this/Inv2  (all ref0:(one User),ref1:(one Work)|(some ref...          5   
 9   this/Inv4  (all ref0:(one User),ref1:(one Work),ref2:(one...          5   
 10  this/Inv3  (all ref0:(one Source),ref1:(one Source),ref2:...          5   
 11  this/Inv4  ((~ ((User <: visi

### Get Min Solutions Data Frames

WARNING: Requires GraphId Maps

In [10]:
client = MongoClient(mongo_uri)
db = client[database_name]

node_collection = db["Node"]

name_to_min_sol_dfs = {}

for (name,graph_ids) in name_to_graph_ids.items():
    data = list(node_collection.aggregate(get_min_solutions_pipeline(graph_ids)))
    df_ = pd.DataFrame(data)
    name_to_min_sol_dfs[name] = df_

client.close()

name_to_min_sol_dfs

{'CV':     predicate                                            formula  frequency  \
 0   this/Inv1  (all ref0:(one User)|((ref0 . (User <: visible...         38   
 1   this/Inv2  (all ref0:(one User)|(((ref0 . (User <: profil...         29   
 2   this/Inv1  (all ref0:(one User),ref1:(one Work)|((ref1 in...         11   
 3   this/Inv1           ((User <: visible) in (User <: profile))         11   
 4   this/Inv3  (all ref0:(one User),ref1:(one Work),ref2:(one...         10   
 5   this/Inv1  (all ref0:(one Work)|(((User <: visible) . ref...          9   
 6   this/Inv3  (all ref0:(one Source),ref1:(one User)|((((((W...          6   
 7   this/Inv3  (all ref0:(one User),ref1:(one (ref0 . (User <...          5   
 8   this/Inv1  (((User <: profile) & (User <: visible)) = (Us...          5   
 9   this/Inv2  (all ref0:(one User)|((ref0 . (User <: profile...          4   
 10  this/Inv2  (all ref0:(one User),ref1:(one (ref0 . (User <...          3   
 11  this/Inv3  (all ref0:(one Sou

## Data Frame Persistance

### Write As Multiple Csvs

General Statistics

In [11]:
graph_stats_df.to_csv(path_or_buf="graph_stats.csv",sep=';',float_format='%g',mode='w', index=False)

Popular Formulas

In [12]:
for (name, df_) in name_to_pop_dfs.items():
    df_.to_csv(path_or_buf=name+".popularity.csv",sep=';',float_format='%g',mode='w', index=False)

Solution Formulas

In [13]:
for (name, df_) in name_to_min_sol_dfs.items():
    df_.to_csv(path_or_buf=name+".solution.csv",sep=';',float_format='%g',mode='w', index=False)

### Write as Sheets of a Single XLSX File

WARNING: Requires Every DataFrame

In [14]:
import xlsxwriter

with pd.ExcelWriter('db_study.xlsx', engine='xlsxwriter') as writer:
    workbook = writer.book
    text_wrap = workbook.add_format({'text_wrap': True, 'valign': 'top'})
    bold = workbook.add_format({'bold': True})
    for name in sorted(list(name_to_model_ids.keys())):
        sheet = workbook.add_worksheet(name=name)

        sheet.set_column(0,0,15)
        sheet.set_column(1,1,100,text_wrap)
        sheet.set_column(2,2,15)
        sheet.set_column(3,3,100,text_wrap)
        sheet.set_column(4,4,27)
        
        row = 0
        sheet.merge_range(row,0,row,len(name_to_pop_dfs[name]),"The 30 most frequent formulas",bold)
        row+=1
        name_to_pop_dfs[name].to_excel(excel_writer=writer,sheet_name=name,startrow=row, index=False)
        row+= name_to_pop_dfs[name].shape[0] + 2
        sheet.merge_range(row,0,row,len(name_to_min_sol_dfs[name]),"The valid formulas ordered by their frequency",bold)
        row+=1
        name_to_min_sol_dfs[name].to_excel(excel_writer=writer,sheet_name=name,startrow=row, index=False)
        row+= name_to_min_sol_dfs[name].shape[0] + 2
    
    graph_stats_df.to_excel(excel_writer=writer,sheet_name="General Statistics", index=False)
    workbook.get_worksheet_by_name('General Statistics').set_column(0,0,30)
    workbook.get_worksheet_by_name('General Statistics').set_column(1,4,20)

# Policy Study

In [15]:
exercise = "dkZH6HJNQNLLDX6Aj" # Social Network
data = pd.read_csv('formulas.csv', delimiter=';')

input_data = None
input_data = pd.read_csv('formulas.csv', delimiter=';').sort_values(by='Predicate')
input_data = input_data.reset_index()

body = dict()

for index, row in input_data.iterrows():
    try:
        body[row['Predicate']].append(row['Formula'])
    except KeyError:
        body[row['Predicate']] = [row['Formula']]

input_data

,index,Predicate,Formula
0,0,inv1,all p: Photo |some u: User | u -> p in posts
1,1,inv1,all p: Photo | p in User . posts
2,2,inv1,"all p: Photo, u: User | p in u . posts"
3,3,inv5,"all i: Influencer, u: User | i in u . follows"
4,4,inv5,all u: User | Influencer in u . follows
5,5,inv5,all u: User | u . follows in Influencer


In [18]:
output=requests.request("POST", "http://localhost:8080/study/test-all-policies-on-formulas?model_id="+exercise, json=body).json()
df = pd.DataFrame(output)
df

,policy,formula,predicate,hint,nextFormula,normalizedFormula
0,MIN-ONE,all p: Photo |some u: User | u -> p in posts,inv1,Keep going! Instead of using inclusion operato...,(all ref0:(one Photo)|(some ((User <: posts) ....,(all ref0:(one Photo)|(some ref1:(one User)|((...
1,MIN-ONE,all p: Photo | p in User . posts,inv1,Keep going! Consider adding a unique quantifie...,(all ref0:(one Photo)|(one (ref0 . (~ (User <:...,(all ref0:(one Photo)|(ref0 in (User . (User <...
2,MIN-ONE,"all p: Photo, u: User | p in u . posts",inv1,Keep going! Consider adding a conjunction oper...,((all ref0:(one Photo)|(ref0 in (User . (User ...,"(all ref0:(one Photo),ref1:(one User)|(ref0 in..."
3,MIN-ONE,"all i: Influencer, u: User | i in u . follows",inv5,Keep going! Consider adding a conjunction oper...,"(all ref0:(one Influencer),ref1:(one User)|((r...","(all ref0:(one Influencer),ref1:(one User)|(re..."
4,MIN-ONE,all u: User | Influencer in u . follows,inv5,Near a solution! Consider adding a difference ...,(all ref0:(one User)|((Influencer - ref0) in (...,(all ref0:(one User)|(Influencer in (ref0 . (U...
...,...,...,...,...,...,...
91,COMPxArrival,all p: Photo | p in User . posts,inv1,Keep going! Consider adding a conjunction oper...,"((all ref0:(one Photo),ref1:(one User),ref2:(o...",(all ref0:(one Photo)|(ref0 in (User . (User <...
92,COMPxArrival,"all p: Photo, u: User | p in u . posts",inv1,Keep going! Instead of using universal quantif...,(all ref0:(one Photo)|(lone (ref0 & (User . (U...,"(all ref0:(one Photo),ref1:(one User)|(ref0 in..."
93,COMPxArrival,"all i: Influencer, u: User | i in u . follows",inv5,Keep going! Consider adding a disjunction oper...,"(all ref0:(one Influencer),ref1:(one User)|((r...","(all ref0:(one Influencer),ref1:(one User)|(re..."
94,COMPxArrival,all u: User | Influencer in u . follows,inv5,Keep going! Consider adding a conjunction oper...,(all ref0:(one User)|((ref0 !in (ref0 . (User ...,(all ref0:(one User)|(Influencer in (ref0 . (U...


In [56]:
condensed_output = dict()
for obj in output:
    try:
        condensed_output[(obj['predicate'],obj['formula'],obj.get('nextFormula', None))][obj['policy']] = True
    except KeyError:
        copy = dict(obj)
        policy = obj['policy']
        copy.pop('policy',None)
        copy[policy] = True
        condensed_output[(obj['predicate'],obj['formula'],obj['nextFormula'])] =copy
    
df = pd.DataFrame(condensed_output.values())
df

,formula,predicate,hint,nextFormula,normalizedFormula,MAX-FREQ,COMPxPOPULARITY,TEDxPOPULARITY,BALANCED-TEDCOMP,MINMAX-TED,...,MIN-TED,TEDxArrival,BALANCED-TEDCOMPxPOPULARITY,MIN-ONE,MINMAX-COMP,MAXIMIN-FREQ,Arrival,POPULARITY,BALANCED-TEDCOMPxArrival,COMPxArrival
0,all p: Photo |some u: User | u -> p in posts,inv1,One step away from the solution! Instead of us...,(all ref0:(one Photo)|(one ref1:(one User)|((r...,(all ref0:(one Photo)|(some ref1:(one User)|((...,True,True,True,True,True,...,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
1,all p: Photo | p in User . posts,inv1,"Keep going! Instead of using var0, try using t...",,(all ref0:(one Photo)|(ref0 in (User . (User <...,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
2,"all p: Photo, u: User | p in u . posts",inv1,Keep going! It seems like the inclusion operat...,(all ref0:(one Photo)|(ref0 in (User . (User <...,"(all ref0:(one Photo),ref1:(one User)|(ref0 in...",True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"all i: Influencer, u: User | i in u . follows",inv5,Near a solution! Instead of using variable of ...,"(all ref0:(one Influencer),ref1:(one User)|(re...","(all ref0:(one Influencer),ref1:(one User)|(re...",True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,all u: User | Influencer in u . follows,inv5,Near a solution! Consider adding a difference ...,(all ref0:(one User)|((Influencer - ref0) in (...,(all ref0:(one User)|(Influencer in (ref0 . (U...,True,True,True,True,True,...,NaN,True,True,True,True,True,True,True,NaN,NaN
5,all u: User | u . follows in Influencer,inv5,Keep going! You can use variables to help spec...,"(all ref0:(one Influencer),ref1:(one User)|((r...",(all ref0:(one User)|((ref0 . (User <: follows...,True,NaN,NaN,True,NaN,...,NaN,NaN,NaN,True,True,True,NaN,True,True,True
6,all p: Photo | p in User . posts,inv1,Keep going! It seems like the inclusion operat...,(Photo in (User . (User <: posts))),(all ref0:(one Photo)|(ref0 in (User . (User <...,NaN,True,True,NaN,NaN,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"all p: Photo, u: User | p in u . posts",inv1,One step away from the solution! Instead of us...,(all ref0:(one Photo)|(one ref1:(one User)|(re...,"(all ref0:(one Photo),ref1:(one User)|(ref0 in...",NaN,True,True,NaN,True,...,True,True,True,NaN,True,NaN,NaN,NaN,NaN,NaN
8,"all i: Influencer, u: User | i in u . follows",inv5,Near a solution! Consider adding a difference ...,"(all ref0:(one Influencer),ref1:(one (User - r...","(all ref0:(one Influencer),ref1:(one User)|(re...",NaN,True,True,NaN,True,...,True,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,all u: User | u . follows in Influencer,inv5,Near a solution! Consider adding a signature o...,(all ref0:(one User)|(Influencer in (ref0 . (U...,(all ref0:(one User)|((ref0 . (User <: follows...,NaN,True,True,NaN,True,...,True,True,True,NaN,NaN,NaN,True,NaN,NaN,NaN


In [57]:
df.sort_values(by='predicate', ascending=True)
df.to_csv("hints.csv", index=False, sep=";")
